#  Unit 2.4a Using Programs with Data, SQLAlchemy
> Using Programs with Data is focused on SQL and database actions.  Part A focuses on SQLAlchemy and an OOP programming style,
- toc: true
- image: /images/python.png
- categories: [python]

# Database and SQLAlchemy
> In this blog we will explore using programs with data, focused on Databases.  We will use SQLite Database to learn more about using Programs with Data.  ***Use Debugging through these examples to examine Objects created in Code***.

- Focus for the lesson will be objects, they have data and depth

- College Board talks about ideas like 
    - Program Usage. "iterative and interactive way when processing information"
    - Managing Data.  "classifying data are part of the process in using programs", "data files in a Table"
    - Insight "insight and knowledge can be obtained from ...  digitally represented information"
    - Filter systems. 'tools for finding information and recognizing patterns"
    - Application. "the preserve has two databases", "an employee wants to count the number of book"

- PBL, Databases, Iterative/OOP
    - Iterative. Refers to a sequence of instructions or code being repeated until a specific end result is achieved
    - OOP. A computer programming model that organizes software design around data, or objects, rather than functions and logic
    - SQL. Structured Query Language, abbreviated as SQL, is a language used in programming, managing, and structuring data 


## Imports and Flask Objects
> Defines and key object creations

- Comment on where you have observed these working?  Provide a defintion of purpose.
    1. Flask app object
    2. SQLAlchemy db object


In [2]:
"""
These imports define the key objects
"""

from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///files/sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()

# Breakpoint means that we want to examine the things which have occurred before the breakpoint
# Flask helps to create the app object
# app.config adds to the app, since they are defined and assigned to it, can be called setters

# This belongs in place where it runs once per project
db.init_app(app)


## Model Definition
> Define columns, initialization, and CRUD methods for users table in sqlite.db

- Comment on these items in the class, purpose and defintion.
    - class User: defining the template the class of Users, class definition template, helps us create objects which are of the type User
    - db.Model inheritance: inherit the ability of the class to be able to use database properties, attributes, and methods
    - _init_ method: can be called the constructor, used to instantiate an object from the User class
    - ```@property```, ```@<column>.setter```
    - setters enable us to change the information of the object
    - getters enable us to get the information of the object
    - create, read, update, delete methods: methods which allow us to be able to alter the database, are executable elements whicha re defined by the class, classes can have attributes and methods

In [7]:
""" database dependencies to support sqlite examples """
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash


''' Tutorial: https://www.sqlalchemy.org/library.html#tutorials, try to get into a Python shell and follow along '''

# Define the User class to manage actions in the 'users' table
# -- Object Relational Mapping (ORM) is the key concept of SQLAlchemy
# -- a.) db.Model is like an inner layer of the onion in ORM
# -- b.) User represents data we want to store, something that is built on db.Model
# -- c.) SQLAlchemy ORM is layer on top of SQLAlchemy Core, then SQLAlchemy engine, SQL
# defining the template the class of Users, class definition template, helps us create objects which are of the type User, template for future object
class Stat(db.Model):
    __tablename__ = 'qbstats'  # table name is plural, class name is singular

    # Define the User schema with "vars" from object
    id = db.Column(db.Integer, primary_key=True)
    _name = db.Column(db.String(255), unique=False, nullable=False)
    _atts = db.Column(db.String(255), unique=False, nullable=False)
    _comps = db.Column(db.String(255), unique=False, nullable=False)
    _yards = db.Column(db.String(255), unique=False, nullable=False)
    _tds = db.Column(db.String(255), unique=False, nullable=False)
    _pimage = db.Column(db.String, unique=False)

    # constructor of a User object, initializes the instance variables within object (self)
    def __init__(self, name, atts, comps, yards, tds, pimage):
        self._name = name    # variables with self prefix become part of the object, 
        self._atts = atts
        self._comps = comps
        self._yards = yards
        self._tds = tds
        self._pimage = pimage

    # a name getter method, extracts name from object
    @property
    def name(self):
        return self._name
    
    # a setter function, allows name to be updated after initial object creation
    @name.setter
    def name(self, name):
        self._name = name
    
    @property
    def atts(self):
        return self._atts
    
    # a setter function, allows name to be updated after initial object creation
    @atts.setter
    def atts(self, atts):
        self._atts = atts

    @property
    def comps(self):
        return self._comps
    
    # a setter function, allows name to be updated after initial object creation
    @comps.setter
    def comps(self, comps):
        self._comps = comps

    @property
    def yards(self):
        return self._yards
    
    # a setter function, allows name to be updated after initial object creation
    @yards.setter
    def yards(self, yards):
        self._yards = yards

    @property
    def tds(self):
        return self._tds
    
    # a setter function, allows name to be updated after initial object creation
    @tds.setter
    def tds(self, tds):
        self._tds = tds

    @property
    def pimage(self):
        return self._pimage
    
    @pimage.setter
    def pimage(self, pimage):
        self._pimage = pimage
    
    # output content using str(object) in human readable form, uses getter
    # output content using json dumps, this is ready for API response
    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "name" : self.name,
            "atts" : self.atts,
            "comps" : self.comps,
            "yards" : self.yards,
            "tds": self.tds,
            "pimage": self.pimage
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, name, atts, comps, yards, tds, pimage):
        """only updates values with length"""
        if len(name) > 0:
            self.name = name
        if len(atts) > 0:
            self.atts = atts
        if len(comps) > 0:
            self.comps = comps
        if len(yards) > 0:
            self.yards = yards
        if len(tds) > 0:
            self.tds = tds
        if len(pimage) > 0:
            self.pimage = pimage
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    

/opt/anaconda3/lib/python3.9/site-packages/flask_sqlalchemy/model.py:121: SAWarning: This declarative base already contains a class with the same class name and module name as __main__.Stat, and will be replaced in the string-lookup table.
  super(BindMetaMixin, cls).__init__(name, bases, d)


## Initial Data
> Uses SQLALchemy db.create_all() to initialize rows into sqlite.db

- Comment on how these work?
    1. Create All Tables from db Object
    2. User Object Constructors
    3. Try / Except 


In [8]:
"""Database Creation and Testing """


# Builds working data for testing
def initStats():
    with app.app_context():
        """Create database and tables"""
        db.create_all()
        """Tester data for table"""
        p1 = Stat(name='Patrick Mahomes', atts='648', comps='435', yards='5250', tds='41', pimage='images/pm.png')
        p2 = Stat(name='Justin Herbert', atts='699', comps='477', yards='4739', tds='25', pimage='images/jh.png')
        p3 = Stat(name='Tom Brady', atts='733', comps='490', yards='4694', tds='25', pimage='images/tb.png')
        p4 = Stat(name='Kirk Cousins', atts='643', comps='424', yards='4547', tds='29', pimage='images/kc.png')
        p5 = Stat(name='Joe Burrow', atts='606', comps='414', yards='4475', tds='35', pimage='images/jb.png')
        p6 = Stat(name='Jared Goff', atts='587', comps='382', yards='4438', tds='29', pimage='images/jg.png')
        p7 = Stat(name='Josh Allen', atts='567', comps='359', yards='4283', tds='35', pimage='images/ja.png')
        p8 = Stat(name='Geno Smith', atts='572', comps='399', yards='4283', tds='30', pimage='images/gs.png')
        p9 = Stat(name='Trevor Lawrence', atts='584', comps='387', yards='4113', tds='25', pimage='images/tl.png')
        p10 = Stat(name='Jalen Hurts', atts='460', comps='306', yards='3701', tds='22', pimage='images/jhurts.png')
        qbs = [p1, p2, p3, p4, p5, p6, p7, p8, p9, p10]
        for qb in qbs:
            try:
                qb.create()
            except IntegrityError:
                '''fails with bad or duplicate data'''
                db.session.remove()
                print(f"Record exists for{qb.name}")
                
initStats()

## Check for given Credentials in users table in sqlite.db
> Use of ORM Query object and custom methods to identify user to credentials uid and password

- Comment on purpose of following
    1. User.query.filter_by
    2. user.password

In [11]:
# SQLAlchemy extracts single user from database matching User ID
def find_by_uid(uid):
    with app.app_context():
        # ORM allows us to do query methods on our data
        # only returns the match to the first match to the database
        user = User.query.filter_by(_uid=uid).first()
    return user # returns user object

# Check credentials by finding user and verify password
def check_credentials(uid, password):
    # query email and return user record
    user = find_by_uid(uid)
    # checks for the user
    if user == None:
        return False
    # checks for the password for the selected user
    if (user.is_password(password)):
        return True
    return False
        
check_credentials("indi", "123qwerty")

True

## Create a new User in table in Sqlite.db
> Uses SQLALchemy and custom user.create() method to add row.

- Comment on purpose of following
    1. user.find_by_uid() and try/except
    2. user = User(...)
    3. user.dob and try/except
    4. user.create() and try/except

In [6]:
# Inputs, Try/Except, and SQLAlchemy work together to build a valid database object
def create():
    # optimize user time to see if uid exists
    uid = input("Enter your user id:")
    user = find_by_uid(uid)
    try:
        print("Found\n", user.read())
        return
    except:
        pass # keep going
    
    # request value that ensure creating valid object
    name = input("Enter your name:")
    password = input("Enter your password")
    
    # Initialize User object before date
    user = User(name=name, 
                uid=uid, 
                password=password
                )
    
    # create user.dob, fail with today as dob
    dob = input("Enter your date of birth 'YYYY-MM-DD'")
    try:
        user.dob = datetime.strptime(dob, '%Y-%m-%d').date()
    except ValueError:
        user.dob = datetime.today()
        print(f"Invalid date {dob} require YYYY-mm-dd, date defaulted to {user.dob}")
           
    # write object to database
    with app.app_context():
        try:
            object = user.create()
            print("Created\n", object.read())
        except:  # error raised if object not created
            print("Unknown error uid {uid}")
        
create()

Created
 {'id': 7, 'name': 'michael jordan', 'uid': 'mike', 'dob': '01-01-1900', 'age': 123}


## Reading users table in sqlite.db
> Uses SQLALchemy query.all method to read data

- Comment on purpose of following
    1. User.query.all
    2. json_ready assignment, google List Comprehension

In [7]:

# SQLAlchemy extracts all users from database, turns each user into JSON
def read():
    with app.app_context():
        table = User.query.all()
    json_ready = [user.read() for user in table] # "List Comprehensions", for each user add user.read() to list
    return json_ready

read()

[{'id': 1,
  'name': 'Thomas Edison',
  'uid': 'toby',
  'dob': '02-11-1847',
  'age': 176},
 {'id': 2,
  'name': 'Nikola Tesla',
  'uid': 'niko',
  'dob': '03-13-2023',
  'age': 0},
 {'id': 3,
  'name': 'Alexander Graham Bell',
  'uid': 'lex',
  'dob': '03-13-2023',
  'age': 0},
 {'id': 4,
  'name': 'Eli Whitney',
  'uid': 'whit',
  'dob': '03-13-2023',
  'age': 0},
 {'id': 5,
  'name': 'Indiana Jones',
  'uid': 'indi',
  'dob': '10-21-1920',
  'age': 102},
 {'id': 6,
  'name': 'Marion Ravenwood',
  'uid': 'raven',
  'dob': '10-21-1921',
  'age': 101},
 {'id': 7,
  'name': 'michael jordan',
  'uid': 'mike',
  'dob': '01-01-1900',
  'age': 123}]

# Hacks
- Add this Blog to you own Blogging site.  In the Blog add notes and observations on each code cell.
- Change blog to your own database.
- Add additional CRUD
    - Add Update functionality to this blog.
    - Add Delete functionality to this blog.